<a href="https://colab.research.google.com/github/GentaKanno/tyushutu_honban/blob/main/%E6%96%87%E5%AD%97%E6%8A%BD%E5%87%BA%E6%9C%AC%E7%95%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

for i in range(1, 50):
    txt_file = f"A1-{i}.txt"
    png_file = f"png/strokes {i}.png"

    if not os.path.exists(txt_file):
        print(f"⚠️ {txt_file} が見つかりません。スキップします。")
        continue



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 19)